In [ ]:
%tensorboard --logdir ./logs/fit

In [1]:
import akshare as ak

df = ak.index_stock_cons('000016')

/hy-tmp/miniconda3/envs/quant/lib/python3.8/site-packages/akshare/__init__.py:2735: UserWarning: 为了支持更多特性，请将 Pandas 升级到 2.1.0 及以上版本！
  warnings.warn(
/hy-tmp/miniconda3/envs/quant/lib/python3.8/site-packages/akshare/__init__.py:2740: UserWarning: 为了支持更多特性，请将 Python 升级到 3.9.0 及以上版本！
  warnings.warn(


In [4]:
import sys
sys.path.append('../')

from database_auto.db_data_downloader.downloader_base import DownloaderBase
import database_auto.database_config as db_config
import sqlite3

db_conn = sqlite3.connect('../database_auto/hh_quant_auto.db')
db_downloader = DownloaderBase(db_conn, db_config)

In [11]:
# db_downloader._download_history_indicator_info('sh.600028', '2024-01-01', '2024-12-31').tail()
db_downloader._download_history_date_factor_info('2024-01-01', '2024-12-31').tail()

,datetime,weekday,day_of_week,day_of_month,month,season
80,2024-05-08,2,Wednesday,8,5,Spring
81,2024-05-09,3,Thursday,9,5,Spring
82,2024-05-10,4,Friday,10,5,Spring
83,2024-05-13,0,Monday,13,5,Spring
84,2024-05-14,1,Tuesday,14,5,Spring


In [8]:
df.tail()

,code,datetime,open,high,low,close,volume,amount,turnover_rate
79,sh.600028,2024-05-07,20.3996,20.6259,20.3349,20.5935,123966317.0,7.862712e+08,0.1305
80,sh.600028,2024-05-08,20.5612,20.9492,20.4966,20.7552,126175777.0,8.116725e+08,0.1329
81,sh.600028,2024-05-09,20.8199,20.9168,20.6259,20.6582,121513339.0,7.794047e+08,0.1279
82,sh.600028,2024-05-10,20.6905,20.8199,20.6259,20.6905,125549135.0,8.048338e+08,0.1322
83,sh.600028,2024-05-13,20.6905,20.8199,20.5289,20.6582,111637147.0,7.152156e+08,0.1175


In [53]:
import baostock as bs
import pandas as pd

# 登陆系统
lg = bs.login()
# 显示登陆返回信息
print('login respond error_code:'+lg.error_code)
print('login respond  error_msg:'+lg.error_msg)

# # 获取上证50成分股
# rs = bs.query_sz50_stocks(date='2010-04-28')

# # 登出系统
# bs.logout()

login success!
login respond error_code:0
login respond  error_msg:success


In [54]:
df = bs.query_stock_basic().get_data()
dd = bs.query_stock_industry().get_data()
if not df.empty and not dd.empty:
    dataframe = df.merge(dd[["code", "industry"]], on=["code"], how='left')
    dataframe = dataframe.rename(
        columns={
            "code": "code",
            "code_name": "code_name",
            "industry": "industry",
            "ipoDate": "in_date",
            "outDate": "out_date",
            "type": "type",
            "status": "status",
        }
    )
    dataframe["industry"] = dataframe["industry"].replace("", "其他")
    dataframe["type"] = dataframe["type"].map(lambda x: {"1": "股票", "2": "指数", "3": "其它", "4": "可转债", "5": "ETF"}.get(x, "其他"))
    dataframe["status"] = dataframe["status"].map(lambda x: {"1": "上市", "2": "退市"}.get(x, "其他"))

In [61]:
dataframe['code'].map(lambda x: x.split('.')[0]).value_counts()

sz    3881
sh    3068
of     971
Name: code, dtype: int64

In [ ]:
dataframe[dataframe['code'].str.startswith(('sh', 'sz'))]['code'].shape

In [8]:
code = 'sh.600356'

df = bs.query_history_k_data_plus(
                code=code,
                fields="code,date,open,high,low,close,volume,amount,turn",
                start_date='2000-01-01',
                end_date='2024-05-14',
                frequency="d",
                adjustflag="1",
            ).get_data()

In [9]:
df.head()

""


In [12]:
df = bs.query_stock_basic().get_data()
dd = bs.query_stock_industry().get_data()
if not df.empty and not dd.empty:
    dataframe = df.merge(dd[['code', 'code_name', 'industry']], on=['code', 'code_name'])

In [21]:
dataframe['industry'].replace("", "其他").value_counts()

其他      889
机械设备    472
化工      425
医药生物    397
电子      342
计算机     265
电气设备    242
汽车      213
公用事业    205
传媒      166
轻工制造    150
建筑装饰    148
有色金属    134
交通运输    126
食品饮料    120
房地产     115
通信      109
纺织服装     94
农林牧渔     94
商业贸易     92
国防军工     89
非银金融     82
建筑材料     73
家用电器     71
采掘       63
银行       41
综合       38
休闲服务     36
钢铁       36
Name: industry, dtype: int64

In [3]:
dd = bs.query_stock_basic().get_data()

In [7]:
dd['code'].map(lambda x: x.split('.')[0]).value_counts()

sz    3881
sh    3068
of     971
Name: code, dtype: int64

In [2]:
df = bs.query_trade_dates(start_date='2013-02-05', end_date='2014-02-05').get_data()
df = df[df["is_trading_day"] == "1"]["calendar_date"]  # 选择交易日(%Y-%m-%d)

In [6]:
pd.DataFrame([{'datetime': '2013-02-05'}])

,datetime
0,2013-02-05


In [15]:
df = bs.query_all_stock(day='2013-02-05').get_data()

In [27]:
dd = bs.query_trade_dates(start_date='2013-02-05', end_date='2014-02-05').get_data().to_records(index=False)

In [24]:
sample = df.head().to_records(index=False)

In [13]:
dd = bs.query_all_stock()

In [2]:
import sqlite3
import pandas as pd

db_conn = sqlite3.connect('../database_auto/hh_quant_auto.db')


In [3]:
code = pd.read_sql_query('select distinct code from hh_quant_history_alpha158_factor_info;', db_conn)

In [4]:
code['code'].tolist()[-1]

'sz.300029'

In [5]:
all_stock_info = pd.read_sql_query('select * from hh_quant_all_stock_info;', db_conn)
all_stock_set = list(sorted(all_stock_info[all_stock_info["code"].str.startswith(("sh", "sz"))]["code"].unique()))

In [9]:
len(all_stock_set)

6949

In [21]:
all_stock_set[5245]

'sz.300030'

In [20]:
from datetime import datetime
datetime.strptime('2024-05-07', "%Y-%m-%d")

datetime.datetime(2024, 5, 7, 0, 0)

In [1]:
import tensorflow as tf
import numpy as np

2024-05-16 01:51:28.847155: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-16 01:51:30.047017: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
# sample_1 = np.array([0,0,0])
sample_2 = np.array([-1e9,-1e9,-1e9])

# print(f"version_1: {tf.nn.softmax(sample_1)}")
print(f"version_2: {tf.nn.softmax(sample_2)}")

version_2: [0.33333333 0.33333333 0.33333333]


In [7]:
test = np.array([1,2,3,-65504.])

tf.nn.softmax(test)

<tf.Tensor: shape=(4,), dtype=float64, numpy=array([0.09003057, 0.24472847, 0.66524096, 0.        ])>

In [8]:
pd.DataFrame({"datetime": [1,2,3,4,5]})

,datetime
0,1
1,2
2,3
3,4
4,5


In [1]:
import sys
sys.path.append('../')

import warnings
warnings.filterwarnings("ignore")

import os
import pandas as pd
import numpy as np
import akshare as ak
import sqlite3
import matplotlib.pyplot as plt
%matplotlib inline

from datetime import datetime
from dateutil.relativedelta import relativedelta
from tqdm import tqdm
from database_auto.db_data_downloader.downloader_base import DownloaderBase
import database_auto.database_config as db_config

pd.options.display.max_rows=None
pd.options.display.max_columns=None

!python --version

db_conn = sqlite3.connect('../database_auto/hh_quant_auto.db')
db_downloader = DownloaderBase(db_conn, db_config)

Python 3.8.10


In [ ]:
dd = db_downloader._download_history_alpha158_factor_info()['code'].unique()

In [5]:
df.head()

,code,datetime,KMID,KLEN,KMID2,KUP,KUP2,KLOW,KLOW2,KSFT,KSFT2,OPEN0,OPEN1,OPEN2,OPEN3,OPEN4,HIGH0,HIGH1,HIGH2,HIGH3,HIGH4,LOW0,LOW1,LOW2,LOW3,LOW4,CLOSE0,CLOSE1,CLOSE2,CLOSE3,CLOSE4,VOLUME0,VOLUME1,VOLUME2,VOLUME3,VOLUME4,ROC5,ROC10,ROC20,ROC30,ROC60,MAX5,MAX10,MAX20,MAX30,MAX60,MIN5,MIN10,MIN20,MIN30,MIN60,MA5,MA10,MA20,MA30,MA60,STD5,STD10,STD20,STD30,STD60,BETA5,BETA10,BETA20,BETA30,BETA60,RSQR5,RSQR10,RSQR20,RSQR30,RSQR60,RESI5,RESI10,RESI20,RESI30,RESI60,QTLU5,QTLU10,QTLU20,QTLU30,QTLU60,QTLD5,QTLD10,QTLD20,QTLD30,QTLD60,TSRANK5,TSRANK10,TSRANK20,TSRANK30,TSRANK60,RSV5,RSV10,RSV20,RSV30,RSV60,IMAX5,IMAX10,IMAX20,IMAX30,IMAX60,IMIN5,IMIN10,IMIN20,IMIN30,IMIN60,IMXD5,IMXD10,IMXD20,IMXD30,IMXD60,CORR5,CORR10,CORR20,CORR30,CORR60,CORD5,CORD10,CORD20,CORD30,CORD60,CNTP5,CNTP10,CNTP20,CNTP30,CNTP60,CNTN5,CNTN10,CNTN20,CNTN30,CNTN60,CNTD5,CNTD10,CNTD20,CNTD30,CNTD60,SUMP5,SUMP10,SUMP20,SUMP30,SUMP60,SUMN5,SUMN10,SUMN20,SUMN30,SUMN60,SUMD5,SUMD10,SUMD20,SUMD30,SUMD60,VMA5,VMA10,VMA20,VMA30,VMA60,VSTD5,VSTD10,VSTD20,VSTD30,VSTD60,WVMA5,WVMA10,WVMA20,WVMA30,WVMA60,VSUMP5,VSUMP10,VSUMP20,VSUMP30,VSUMP60,VSUMN5,VSUMN10,VSUMN20,VSUMN30,VSUMN60,VSUMD5,VSUMD10,VSUMD20,VSUMD30,VSUMD60
